In [ ]:
import numpy as np
from astropy.io import fits
from astropy import coordinates as coord
from astropy.modeling import models
from astropy import units as u
from astropy import wcs
import matplotlib.pyplot as plt

In [ ]:
#from stwcs import updatewcs
#updatewcs.updatewcs('ib6o23rsq_flt.fits')

In [ ]:
from astropy.utils.data import download_file
fn = download_file('https://github.com/npirzkal/aXe_WFC3_Cookbook/raw/main/cookbook_data/G141/ib6o23rsq_flt.fits', cache=True)
f = fits.open(fn)
f.info()

In [ ]:
w=wcs.WCS(f[1].header)

In [ ]:
acoef = dict(f[1].header['A*'])
a_order = acoef.pop('A_ORDER')
bcoef = dict(f[1].header['B_*'])
b_order = bcoef.pop('B_ORDER')
crpix = [f[1].header['CRPIX1'], f[1].header['CRPIX2']]

sip = models.SIP(np.array(crpix)-1, a_order=a_order, a_coeff=acoef, b_order=b_order, b_coeff=bcoef)

print('sip:', sip(1,1))
print('\nSIP computes the changes relative to (0,0), '
      'while astropy.wcs computes the sip correction relative to crpix. So we need to add '
      'CRPIX-1 (0-based) to the wcs solution in order to get what SIP computes.\n')
print('sip_wcs:', w.sip_pix2foc(1, 1,0) + np.array((506, 506)))
print('crpix :', crpix)

In [ ]:
crval = [f[1].header['CRVAL1'], f[1].header['CRVAL2']]
cdmat = np.array([[f[1].header['CD1_1'], f[1].header['CD1_2']],
                  [f[1].header['CD2_1'], f[1].header['CD2_2']]])
cdmat

In [ ]:
model = sip | models.Shift(-(crpix[0]-1)) & models.Shift(-(crpix[1]-1)) | \
        models.AffineTransformation2D(matrix=cdmat) | models.Pix2Sky_TAN() | \
        models.RotateNative2Celestial(crval[0], crval[1], 180)
sip(0,0)

In [ ]:
w.all_pix2world(1,1,0)

In [ ]:
# This is the Linear WCS only, without distortion
print('crpix:', w.wcs_pix2world(crpix[0], crpix[1],1))
print('1,1: ', w.wcs_pix2world(1,1,1))

m = models.Shift(-crpix[0]) & models.Shift(-crpix[1]) | \
    models.AffineTransformation2D(matrix=cdmat) | models.Pix2Sky_TAN() | \
    models.RotateNative2Celestial(crval[0], crval[1], 180)
print('crpix:', m(crpix[0], crpix[1]))
print('1,1:', m(1,1))

In [ ]:
apcoef = {}
for key in acoef:
    apcoef['c' + key.split('A_')[1]] = acoef[key]
    
bpcoef = {}
for key in bcoef:
    bpcoef['c' + key.split('B_')[1]] = bcoef[key]
    
ap = models.Polynomial2D(a_order, **apcoef)
bp = models.Polynomial2D(b_order, **bpcoef)

In [ ]:
# And with regular Polynomials instead of SIP
mp = models.Shift(-crpix[0]-1) & models.Shift(-crpix[1]-1) | \
     models.Mapping((0, 1, 0, 1)) | ap & bp | \
     models.AffineTransformation2D(matrix=cdmat) | models.Pix2Sky_TAN() |\
     models.RotateNative2Celestial(crval[0], crval[1], 180)
model(0, 0 )

In [ ]:
w.all_pix2world(1,1,0)

In [ ]:
## Comparing with plots as Iva did

xx = np.arange(0,1014, 20)
yy = np.arange(0,1014, 20)

astropy_coords = w.pixel_to_world(xx, yy)
ra_sip, dec_sip = model(xx, yy)
ra_poly, dec_poly = mp(xx, yy)

In [ ]:
ra_sip-ra_poly

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(10,10))

axs[0,0].plot(xx, ra_sip)
axs[0,0].plot(xx, ra_poly)
axs[0,0].plot(xx, astropy_coords.ra.value)

axs[0,1].plot(xx, ra_sip-astropy_coords.ra.value, 'o', color='black', markersize=0.5)
axs[0,1].plot(xx, ra_sip-ra_poly, 'o', color='blue', markersize=0.5)

axs[1,0].plot(yy, dec_sip)
axs[1,0].plot(yy, dec_poly)
axs[1,0].plot(yy, astropy_coords.dec.value)

axs[1,1].plot(yy, dec_sip-astropy_coords.dec.value,  'o', color='black', markersize=0.5)
axs[1,1].plot(yy, dec_sip-dec_poly,  'o', color='blue', markersize=0.5)

In [ ]:
m(xx, yy)

In [ ]:
print(model(507, 507))
print(m(507, 507))
print(mp(507, 507))
print(w.pixel_to_world(507,507))
print(w.all_pix2world(507, 507, 0))

In [ ]:
f[1].header

In [ ]:
ra_diff = ra_poly-astropy_coords.ra.value
ra_diff

In [ ]:
ra_diff[1:] - ra_diff[:-1]

In [ ]:
cdmat[0,0]*20 + cdmat[0,1]*20

In [ ]:
cdmat[0,1]

In [ ]:
model

In [ ]:
## Ricky's attempt at defining a model that matches the astropy WCS calculations

# Repeating coefficient retrievals here
acoef = dict(f[1].header['A*'])
a_order = acoef.pop('A_ORDER')
bcoef = dict(f[1].header['B_*'])
b_order = bcoef.pop('B_ORDER')
crpix = [f[1].header['CRPIX1'], f[1].header['CRPIX2']]

mr = (models.Shift(-(crpix[0]-1)) & models.Shift(-(crpix[1]-1)) | # Calculate u and v coords
     models.Mapping((0, 1, 0, 1, 0, 1)) | ap & bp & models.Identity(2) | # calculate f(u,v) and g(u,v)
     models.Mapping((0, 2, 1, 3)) | models.math.AddUfunc() & models.math.AddUfunc() | # Calculate u+f(u,v) and v+g(u,v)
     models.AffineTransformation2D(matrix=cdmat) | models.Pix2Sky_TAN() | 
     models.RotateNative2Celestial(crval[0], crval[1], 180)
     )
mr(0, 0)

In [ ]:
ra_r, dec_r = mr(xx, yy)

fig, axs = plt.subplots(2,2, figsize=(10,10))

axs[0,0].plot(xx, ra_r)
axs[0,0].plot(xx, astropy_coords.ra.value)

axs[0,1].plot(xx, ra_r-astropy_coords.ra.value, 'o', color='black', markersize=0.5)

axs[1,0].plot(yy, dec_r)
axs[1,0].plot(yy, astropy_coords.dec.value)

axs[1,1].plot(yy, dec_r-astropy_coords.dec.value,  'o', color='black', markersize=0.5)